#### v1


In [14]:
!pip install --upgrade pip


  Using cached pip-25.0.1-py3-none-any.whl (1.8 MB)


ERROR: To modify pip, please run the following command:
C:\Python311\python.exe -m pip install --upgrade pip

[notice] A new release of pip is available: 23.1.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [19]:
!pip install  langchain-text-splitters langchain-community langgraph

  Using cached langchain_community-0.3.17-py3-none-any.whl.metadata (2.4 kB)
  Using cached langchain-0.3.18-py3-none-any.whl.metadata (7.8 kB)
Using cached langchain_community-0.3.17-py3-none-any.whl (2.5 MB)
Using cached langchain-0.3.18-py3-none-any.whl (1.0 MB)
  Attempting uninstall: langchain
    Found existing installation: langchain 0.2.1
    Uninstalling langchain-0.2.1:
      Successfully uninstalled langchain-0.2.1
  Rolling back uninstall of langchain
  Moving to c:\python311\lib\site-packages\langchain-0.2.1.dist-info\
   from C:\Python311\Lib\site-packages\~angchain-0.2.1.dist-info
  Moving to c:\python311\lib\site-packages\langchain\
   from C:\Python311\Lib\site-packages\~angchain


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'C:\\Python311\\Scripts\\langchain-server.exe' -> 'C:\\Python311\\Scripts\\langchain-server.exe.deleteme'



In [20]:
pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [21]:
pip install -qU langchain-openai

Note: you may need to restart the kernel to use updated packages.


ERROR: Could not install packages due to an OSError: [WinError 2] The system cannot find the file specified: 'c:\\Python311\\Scripts\\openai.exe' -> 'c:\\Python311\\Scripts\\openai.exe.deleteme'



In [22]:
!pip install rouge-score nltk


In [6]:
import getpass
import os
import pymongo
import pprint
import pandas as pd
from sklearn.metrics import f1_score
from nltk.translate.bleu_score import sentence_bleu
from rouge_score import rouge_scorer
from nltk.translate.meteor_score import meteor_score
import nltk
from langchain_community.document_loaders import PyPDFLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_mongodb import MongoDBAtlasVectorSearch
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pymongo import MongoClient
from langchain_community.document_loaders import DirectoryLoader

ModuleNotFoundError: No module named 'pymongo'

In [ ]:
# Ensure NLTK resources are downloaded
nltk.download('wordnet')
nltk.download('omw-1.4')

In [ ]:
import getpass
import os

if not os.environ.get("OPENAI_API_KEY"):
  os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter API key for OpenAI: ")

from langchain_openai import OpenAIEmbeddings

embeddings = OpenAIEmbeddings(model="text-embedding-3-large")

Enter API key for OpenAI: ··········


In [ ]:
pip install -qU langchain-mongodb

In [ ]:
!pip install pymongo pypdf

In [ ]:
import getpass

MONGODB_ATLAS_CLUSTER_URI = getpass.getpass("MongoDB Atlas Cluster URI:")

MongoDB Atlas Cluster URI:··········


In [ ]:
from pymongo import MongoClient
from pymongo.operations import SearchIndexModel

In [ ]:
# mongodb Atlas cluster connection
client = MongoClient(MONGODB_ATLAS_CLUSTER_URI)

# mongodb collection and index name
db_name = "v1_copilot"
collection_name = "v2"
atlas_collection = client[db_name][collection_name]
vector_search_index = "copilot_index"

In [ ]:
!pip install unstructured

In [ ]:
# Load the PDF
folder_path = "/content/drive/MyDrive/downloaded_documents"
# Instead of using '../', use the actual folder path
loader = DirectoryLoader(folder_path, glob="**/*.pdf", show_progress=True , loader_cls = PyPDFLoader)
# loader = PyPDFLoader("https://arxiv.org/pdf/2501.01478")
data = loader.load()

# Split PDF into documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=600, chunk_overlap=50)
docs = text_splitter.split_documents(data)

# Print the first document
docs[0]

In [ ]:
# Create the vector store
vector_store = MongoDBAtlasVectorSearch.from_documents(
    documents = docs,
    embedding = OpenAIEmbeddings(disallowed_special=()),
    collection = atlas_collection,
    index_name = vector_search_index
)

In [ ]:
# Use helper method to create the vector search index
vector_store.create_vector_search_index(
   dimensions = 1536,
   filters = [ "page" ]
)

In [ ]:
# Instantiate Atlas Vector Search as a retriever

# #### expriment around retriever
  #### setup RAG evals using RAGAS/custom
  #### create test dataset
# retriever = vector_store.as_retriever(
#    search_type = "similarity",
#    search_kwargs = { "k": 15 }
# )
retriever = MongoDBAtlasVectorSearch(
    embedding=OpenAIEmbeddings(),
    collection=atlas_collection,
    index_name=vector_search_index,
    relevance_score_fn="cosine",
    search_type = "similarity",
    search_kwargs = { "k": 15 }
).as_retriever()

#prompt template
template = """ You are a kind and helpful QnA assistant. Your job is to assist Faculty and students of BITS Pilani in resolving their doubts and queries.
   {context}
   Question: {question}
"""
prompt = PromptTemplate.from_template(template)
model = ChatOpenAI(openai_api_key = "")

# llm chain
chain = (
   { "context": retriever, "question": RunnablePassthrough()}
   | prompt
   | model
   | StrOutputParser()
)

# Prompt the chain
# question = """What are the guidelines for availing casual leaves by a PhD student?"""
# answer = chain.invoke(question)

# print("Question: " + question)
# print("Answer: " + answer)

# # Return source documents
# documents = retriever.invoke(question)
# print("\nSource documents:")
# pprint.pprint(documents)

In [ ]:
# Define evaluation functions
def exact_match_score(pred, truth):
    return int(pred.strip().lower() == truth.strip().lower())

def f1_score_token(pred, truth):
    pred_tokens = pred.strip().lower().split()
    truth_tokens = truth.strip().lower().split()
    common_tokens = set(pred_tokens) & set(truth_tokens)
    if len(common_tokens) == 0:
        return 0.0
    precision = len(common_tokens) / len(pred_tokens)
    recall = len(common_tokens) / len(truth_tokens)
    f1 = 2 * (precision * recall) / (precision + recall)
    return f1

def bleu_score(pred, truth):
    pred_tokens = pred.strip().lower().split()
    truth_tokens = [truth.strip().lower().split()]
    return sentence_bleu(truth_tokens, pred_tokens)

def rouge_score(pred, truth):
    scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
    scores = scorer.score(truth.strip().lower(), pred.strip().lower())
    return scores['rougeL'].fmeasure

def meteor_score_custom(pred, truth):
    return meteor_score([truth.strip().lower().split()], pred.strip().lower().split())

# Load the CSV file containing question-answer pairs
csv_path = "/content/drive/MyDrive/IR_test_dataset.csv"  # Replace with your CSV file path
df = pd.read_csv(csv_path)

# Initialize a list to store results
results = []

# Iterate over each question-answer pair
for index, row in df.iterrows():
    question = row["Question"]
    ground_truth_answer = row["Answer"]

    # Generate answer using the RAG system
    generated_answer = chain.invoke(question)

    # Evaluate the generated answer
    em = exact_match_score(generated_answer, ground_truth_answer)
    f1 = f1_score_token(generated_answer, ground_truth_answer)
    bleu = bleu_score(generated_answer, ground_truth_answer)
    rouge = rouge_score(generated_answer, ground_truth_answer)
    meteor = meteor_score_custom(generated_answer, ground_truth_answer)

    # Store results
    results.append({
        "question": question,
        "ground_truth_answer": ground_truth_answer,
        "generated_answer": generated_answer,
        "exact_match": em,
        "f1_score": f1,
        "bleu_score": bleu,
        "rouge_score": rouge,
        "meteor_score": meteor
    })

# Convert results to a DataFrame
results_df = pd.DataFrame(results)

# Save results  CSV
results_df.to_csv("/content/drive/MyDrive/IR_Project/v2_index.csv", index=False)

# Printscores
print("Average Scores:")
print(f"Exact Match: {results_df['exact_match'].mean()}")
print(f"F1 Score: {results_df['f1_score'].mean()}")
print(f"BLEU Score: {results_df['bleu_score'].mean()}")
print(f"ROUGE Score: {results_df['rouge_score'].mean()}")
print(f"METEOR Score: {results_df['meteor_score'].mean()}")